In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\kidney_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\kidney_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file:Path
    unzip_dir:Path

In [6]:

from KidneyCNN.constants import *
from KidneyCNN.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [8]:
import os
import zipfile
import gdown
from KidneyCNN import logger
from KidneyCNN.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self)-> str:
        # The file will only be download if it dosn't already exist
        if not os.path.exists(self.config.local_data_file):
            logger.info("Starting file download...")
            '''
            Fetch data from url 
            '''
            try:
                dataset_url = self.config.source_URL
                Zip_download_dir = self.config.local_data_file
                os.makedirs("artifacts/data_ingestion", exist_ok=True)
                logger.info(f"Downloading data from {dataset_url} into file {Zip_download_dir}")

                file_id= dataset_url.split("/")[-2]
                prefix = "https://drive.google.com/uc?/export=download&id="
                gdown.download(prefix+file_id, Zip_download_dir)

                logger.info(f"Downloading data from {dataset_url} into file {Zip_download_dir}")

            except Exception as e:
                raise e
        

        else:
            # Original logic for file already existing
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
            Zip_file_path: str
            Extract the zip file in to data diractory
            Function return None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-10-03 23:49:22,542: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-03 23:49:22,547: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-03 23:49:22,550: INFO: common: created directory at: artifacts]
[2025-10-03 23:49:22,552: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-03 23:49:22,553: INFO: 286326510: File already exists of size: ~ 106843 KB]
